In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import os
from pathlib import Path

In [2]:
data=pd.read_csv(r'C:\Users\USER\Desktop\Inua360-The-Kenyan-SME-AI-Agent\data\interim\Clean_data.csv')
data.columns = data.columns.str.strip()
data.head()

,company_id,country,sector,employees,annual_revenue,tech_adoption_level,main_challenges,digital_tools_used,growth_last_yr,funding_status,female_owned,remote_work_policy
0,1,Ghana,Education,130,386441,Low,Awareness,"CRM, WhatsApp, E-commerce",11,Seed,No,No Policy
1,2,Rwanda,Farming,367,383576,Low,Internet,WhatsApp,27,Series A,Yes,Partial
2,3,Kenya,Farming,87,496528,Low,"Awareness, Internet","Google My Business, Mobile money, POS",1,Bootstrapped,No,Full
3,4,Kenya,Manufacturing,131,360550,High,Regulation,"Google My Business, E-commerce, POS",6,Unknown,Yes,Full
4,5,Nigeria,Logistics,136,173801,High,"Regulation, Skills",E-commerce,44,Series A,Yes,Partial


In [3]:
data.columns = data.columns.str.strip()

In [4]:
data.columns.to_list()

['company_id',
 'country',
 'sector',
 'employees',
 'annual_revenue',
 'tech_adoption_level',
 'main_challenges',
 'digital_tools_used',
 'growth_last_yr',
 'funding_status',
 'female_owned',
 'remote_work_policy']

In [5]:
data['revenue_per_employee'] = data['annual_revenue']/data['employees']
data['revenue_per_employee'] = data['revenue_per_employee'].replace([np.inf, -np.inf], np.nan)
data.head()

,company_id,country,sector,employees,annual_revenue,tech_adoption_level,main_challenges,digital_tools_used,growth_last_yr,funding_status,female_owned,remote_work_policy,revenue_per_employee
0,1,Ghana,Education,130,386441,Low,Awareness,"CRM, WhatsApp, E-commerce",11,Seed,No,No Policy,2972.623077
1,2,Rwanda,Farming,367,383576,Low,Internet,WhatsApp,27,Series A,Yes,Partial,1045.166213
2,3,Kenya,Farming,87,496528,Low,"Awareness, Internet","Google My Business, Mobile money, POS",1,Bootstrapped,No,Full,5707.218391
3,4,Kenya,Manufacturing,131,360550,High,Regulation,"Google My Business, E-commerce, POS",6,Unknown,Yes,Full,2752.290076
4,5,Nigeria,Logistics,136,173801,High,"Regulation, Skills",E-commerce,44,Series A,Yes,Partial,1277.948529


In [6]:
'''if a company is tiny and has low technology use, then it's compliance risk is high and if it has ten to fifteen employees we assume medium risk and larger
companies with better digital tools are considered low risk'''

def compliance_risk(row):
    if row['employees'] < 10 and row["tech_adoption_level"] in ["Low", "None"]:
        return "High"
    elif 10 <= row["employees"] <= 50:
        return "Medium"
    else:
        return "Low"

#applies that function to evert row and createa a new column which is now the taret variaable for the risk model 
data['Compliance_risk'] = data.apply(compliance_risk, axis = 1)


data.head()

,company_id,country,sector,employees,annual_revenue,tech_adoption_level,main_challenges,digital_tools_used,growth_last_yr,funding_status,female_owned,remote_work_policy,revenue_per_employee,Compliance_risk
0,1,Ghana,Education,130,386441,Low,Awareness,"CRM, WhatsApp, E-commerce",11,Seed,No,No Policy,2972.623077,Low
1,2,Rwanda,Farming,367,383576,Low,Internet,WhatsApp,27,Series A,Yes,Partial,1045.166213,Low
2,3,Kenya,Farming,87,496528,Low,"Awareness, Internet","Google My Business, Mobile money, POS",1,Bootstrapped,No,Full,5707.218391,Low
3,4,Kenya,Manufacturing,131,360550,High,Regulation,"Google My Business, E-commerce, POS",6,Unknown,Yes,Full,2752.290076,Low
4,5,Nigeria,Logistics,136,173801,High,"Regulation, Skills",E-commerce,44,Series A,Yes,Partial,1277.948529,Low


In [7]:
categorical_cols=data.select_dtypes(include='object')
categorical_cols.columns.to_list()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    data[col]=le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

data.nunique()

company_id              1000
country                    5
sector                     6
employees                421
annual_revenue          1000
tech_adoption_level        3
main_challenges           25
digital_tools_used       149
growth_last_yr            56
funding_status             4
female_owned               2
remote_work_policy         3
revenue_per_employee    1000
Compliance_risk            3
dtype: int64

In [ ]:
funding_status=data['funding_status']

data=data.drop('funding_status', axis=1)

data.head()



,company_id,country,sector,employees,annual_revenue,tech_adoption_level,main_challenges,digital_tools_used,growth_last_yr,funding_status,female_owned,remote_work_policy,revenue_per_employee,Compliance_risk
0,1,0,0,130,386441,1,0,20,11,1,0,1,2972.623077,1
1,2,3,1,367,383576,1,10,124,27,2,1,2,1045.166213,1
2,3,1,1,87,496528,1,2,61,1,0,0,0,5707.218391,1
3,4,1,4,131,360550,0,15,56,6,3,1,0,2752.290076,1
4,5,2,3,136,173801,0,19,24,44,2,1,2,1277.948529,1


In [9]:
data.columns.to_list()

['company_id',
 'country',
 'sector',
 'employees',
 'annual_revenue',
 'tech_adoption_level',
 'main_challenges',
 'digital_tools_used',
 'growth_last_yr',
 'funding_status',
 'female_owned',
 'remote_work_policy',
 'revenue_per_employee',
 'Compliance_risk']